# DSCI 552 Homework 3

Name: Ziyue Chen

Github Username: zic115

USC ID: 8823457220

# 1. Time Series Classification Part 1: Feature Creation/Extraction

In [1]:
# Import packages
import pandas as pd
import numpy as np
from scipy.stats import bootstrap

## 1. (b) Generate train and test data

In [2]:
# Generate file paths
folders = {'bending1': 7,
           'bending2': 6,
           'cycling': 15,
           'lying': 15,
           'sitting': 15,
           'standing': 15,
           'walking': 15}

paths = []
for folder in folders:
    for file in range(1, folders[folder] + 1):
        path = '../data/AReM/' + folder + '/dataset' + str(file) + '.csv'
        paths.append(path)

In [3]:
# Split train and test paths
test_paths = ['../data/AReM/bending1/dataset1.csv',
              '../data/AReM/bending1/dataset2.csv',
              '../data/AReM/bending2/dataset1.csv',
              '../data/AReM/bending2/dataset2.csv',
              '../data/AReM/cycling/dataset1.csv',
              '../data/AReM/cycling/dataset2.csv',
              '../data/AReM/cycling/dataset3.csv',
              '../data/AReM/lying/dataset1.csv',
              '../data/AReM/lying/dataset2.csv',
              '../data/AReM/lying/dataset3.csv',
              '../data/AReM/sitting/dataset1.csv',
              '../data/AReM/sitting/dataset2.csv',
              '../data/AReM/sitting/dataset3.csv',
              '../data/AReM/standing/dataset1.csv',
              '../data/AReM/standing/dataset2.csv',
              '../data/AReM/standing/dataset3.csv',
              '../data/AReM/walking/dataset1.csv',
              '../data/AReM/walking/dataset2.csv',
              '../data/AReM/walking/dataset3.csv']

train_paths = []
for i in range(len(paths)):
    if paths[i] not in test_paths:
        train_paths.append(paths[i])

In [4]:
# Generate test set
test = None
for path in test_paths:
    df = pd.read_csv(path, skiprows = 4)
    test = pd.concat([test, df])
    
test

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.30
1,250,39.25,0.43,23.00,0.00,33.00,0.00
2,500,39.25,0.43,23.25,0.43,33.00,0.00
3,750,39.50,0.50,23.00,0.71,33.00,0.00
4,1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...,...
475,118750,36.00,2.45,17.00,5.10,20.50,0.87
476,119000,34.33,1.89,15.00,2.45,17.00,2.12
477,119250,33.00,7.35,14.60,3.14,13.00,5.70
478,119500,31.67,1.25,11.00,6.16,19.25,2.17


In [5]:
# Generate train set
train = None
for path in train_paths:
    df = pd.read_csv(path, skiprows = 4)
    train = pd.concat([train, df])
    
train

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43
...,...,...,...,...,...,...,...
475,118750,34.50,6.18,9.00,3.56,12.67,4.19
476,119000,25.75,6.02,13.75,2.05,16.00,1.58
477,119250,31.50,3.35,10.25,5.12,16.25,2.95
478,119500,33.75,2.77,14.00,3.24,13.75,0.43


## 1. (c) Feature Extraction

### 1. (c) i. Time-domain features that are usually used in time series classification

Some common features are minimum, maximum, mean, standard deviation, and skewness.

### 1. (c) ii. Extract the time-domain features for all of the 6 time series in each instance

In [6]:
# Define a function for extraction
def extractFeatures(df, instance):
    data = {'Instance': [instance]}
    time_series = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
    
    for i in range(1, 7):
        curr = time_series[i - 1]
        minimum = df[curr].min()
        maximum = df[curr].max()
        mean = df[curr].mean()
        median = df[curr].median()
        std = df[curr].std()
        q1 = df[curr].quantile(0.25)
        q3 = df[curr].quantile(0.75)
        
        curr_data = {'Min_' + str(i): [minimum],
                     'Max_' + str(i): [maximum],
                     'Mean_' + str(i): [mean],
                     'Median_' + str(i): [median],
                     'Std_' + str(i): [std],
                     'Q1_' + str(i): [q1],
                     'Q3_' + str(i): [q3]}
        data = data | curr_data

    stats = pd.DataFrame(data)
    return stats

In [7]:
# Extract features from each instance
res = None
for i in range(len(paths)):
    df = pd.read_csv(paths[i], skiprows = 4)
    stats = extractFeatures(df, i + 1)
    res = pd.concat([res, stats])

res.reset_index(drop = True)

,Instance,Min_1,Max_1,Mean_1,Median_1,Std_1,Q1_1,Q3_1,Min_2,Max_2,...,Std_5,Q1_5,Q3_5,Min_6,Max_6,Mean_6,Median_6,Std_6,Q1_6,Q3_6
0,1,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582915,0.0000,1.3000
1,2,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,...,1.995255,32.0000,34.50,0.00,3.11,0.571083,0.43,0.601010,0.0000,1.3000
2,3,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,...,1.999604,35.3625,36.50,0.00,1.79,0.493292,0.43,0.513506,0.0000,0.9400
3,4,33.00,47.75,42.179813,43.50,3.670666,39.15,45.0000,0.0,3.00,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.50,0.524317,0.0000,1.0000
4,5,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.43,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,19.50,45.33,33.586875,34.25,4.650935,30.25,37.0000,0.0,14.67,...,3.283983,13.7300,18.25,0.00,8.32,3.259729,3.11,1.640243,2.0500,4.3225
84,85,19.75,45.50,34.322750,35.25,4.752477,31.00,38.0000,0.0,13.47,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.20,1.732727,2.1575,4.5650
85,86,19.50,46.00,34.546229,35.25,4.842294,31.25,37.8125,0.0,12.47,...,2.823124,14.0000,17.75,0.00,10.00,3.338125,3.08,1.656742,2.1600,4.3350
86,87,23.50,46.25,34.873229,35.25,4.531720,31.75,38.2500,0.0,14.82,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.27,1.690960,2.1700,4.5000


### 1. (c) iii. Standard deviation of each of the time-domain features

In [8]:
# Calculate the standard deviations
columns = list(res.columns[1:])
stds = None
for col in columns:
    std = res[col].std()
    stats = pd.DataFrame({'Std': [std]}).rename(index = {0: col})
    stds = pd.concat([stds, stats])

stds

,Std
Min_1,9.569975
Max_1,4.394362
Mean_1,5.335718
Median_1,5.440054
Std_1,1.772153
Q1_1,6.153590
Q3_1,5.138925
Min_2,0.000000
Max_2,5.062729
Mean_2,1.574164


In [9]:
# Build a 90% bootsrap confidence interval for the standard deviation of each feature
for col in columns:
    data = (res[col], )
    CI = bootstrap(data, np.std, confidence_level = 0.9, method = 'basic')
    print(col + ':', CI.confidence_interval)

Min_1: ConfidenceInterval(low=8.337983669713466, high=10.82045465092324)
Max_1: ConfidenceInterval(low=3.4837800050072456, high=5.425429439096328)
Mean_1: ConfidenceInterval(low=4.75746862748623, high=5.926826361628337)
Median_1: ConfidenceInterval(low=4.859097788398155, high=6.044403998226757)
Std_1: ConfidenceInterval(low=1.5822215182940507, high=1.9626955662110517)
Q1_1: ConfidenceInterval(low=5.637791476532069, high=6.691140603338468)
Q3_1: ConfidenceInterval(low=4.416610309114545, high=5.917896432393834)
Min_2: ConfidenceInterval(low=0.0, high=0.0)
Max_2: ConfidenceInterval(low=4.689211022003796, high=5.470545367964498)
Mean_2: ConfidenceInterval(low=1.4307278914903638, high=1.741087209199352)
Median_2: ConfidenceInterval(low=1.2731259663533006, high=1.5775408456053734)
Std_2: ConfidenceInterval(low=0.8219756547342562, high=0.9618201498623573)
Q1_2: ConfidenceInterval(low=0.8513139282529298, high=1.0549480138087868)
Q3_2: ConfidenceInterval(low=1.945691012944284, high=2.3425683106

### 1. (c) iv. Select the three most important time-domain features

Three most important time-domain features can be 1st quartile, median, and 3rd quartile.

# 2. ISLR 3.7.4

## 2. (a) True relationship is linear: training RSS for linear vs. training RSS for cubic

The training RSS for cubic should be lower than the training RSS for linear because the cubic regression has more polynomial terms and a tighter fit to the data.

## 2. (b) True relationship is linear: test RSS for linear vs. test RSS for cubic

The test RSS for linear should be lower than the test RSS for cubic because the true relationship is linear and the cubic regression can overfit to the data.

## 2. (c) True relationship is not linear: training RSS for linear vs. training RSS for cubic

The training RSS for cubic should be lower than the training RSS for linear because the cubic regression has more polynomial terms and a higher flexibility to fit to the non-linear data.

## 2. (d) True relationship is not linear: test RSS for linear vs. test RSS for cubic

There is not enough information to tell because non-linear data doesn't necessarily mean it's cubic. If the true relationship is only slightly different from linear, then the test RSS for linear should be lower than the test RSS for cubic. If the true relationship is similar to cubic, then the test RSS for cubic should be lower.
